## Read from Da-TACOS feature-fused SSMs and compute shape DNA and hierarchical structure decomposition

### Library importing

In [11]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
from scipy.interpolate import interp2d
from scipy.sparse.csgraph import laplacian
from scipy.spatial.distance import directed_hausdorff
from scipy.cluster import hierarchy
from scipy.linalg import eigh
from scipy.ndimage import median_filter
import cv2
from seaborn import clustermap
import sklearn
import librosa
import glob
import os
import random
import json
import deepdish as dd
import matplotlib.pyplot as plt
%matplotlib inline

### Reading and computing structure

In [18]:
with open('./da-tacos_metadata/da-tacos_benchmark_subset_metadata.json') as f:
    benchmark_metadata = json.load(f)

count=0
#dict = {} #everything stored here keeping W,P hierarchy, as [shapeDNA, dist_set]
all_shapeDNA = []
all_distset = []
for W in benchmark_metadata.keys():
    #W_dict = {}
    W_list = []
    y = []
    for P in benchmark_metadata[W].keys():
        #Computations
        try:
            SSM = dd.io.load("./da-tacosSSMs/StructureLaplacian_datacos_crema_" + P + ".h5")['WFused']
        except:
            continue
        else:
            N = dd.io.load("./da-tacosSSMs/StructureLaplacian_datacos_crema_" + P + ".h5")['N']

            #Construct square matrix from flattened upper triangle
            A = np.zeros((N,N))
            iN = np.triu_indices(N) #return indices for upper-triangle of (N,N) matrix
            for i in range(len(SSM)):
                A[iN[0][i]][iN[1][i]] = SSM[i]
            B = np.transpose(A)
            square_SSM = A+B

            #Downsample to 256X256   
            Xindex = np.linspace(0, 1, num=N)
            f = interp2d(Xindex, Xindex, square_SSM.flatten(), kind='linear')
            Xindex_ds = np.linspace(0, 1, num=256)
            SSM_ds = np.reshape(f(Xindex_ds, Xindex_ds), (256,256))
            
            #Compute the Laplacian
            L = laplacian(SSM_ds, normed=True)
            
            #Laplacian eigenvalues and eigenvectors
            evals, evecs = eigh(L)
            
            #Shape DNA
            shapeDNA = evals[:30]
            
            #Hierarchical structure
            evecs = median_filter(evecs, size=(9, 1))
            Cnorm = np.cumsum(evecs**2, axis=1)**0.5
            dist_set = []
            for k in range(2, 10): #change range here (min value 2)
                X = evecs[:, :k] / Cnorm[:, k-1:k]
                distance = squareform(pdist(X, metric='euclidean'))
                dist_set.append(distance)
            
            #W_dict[P] = [shapeDNA, dist_set]
            all_shapeDNA.append(shapeDNA)
            all_distset.append(dist_set)
            y.append(W)
            
            #plt.matshow()
            #plt.colorbar()
            #plt.show()
            
            count+=1       
    #dict[W] = W_dict 
    
    if (count >= 100):
        break